<div style="text-align: center;">
<h1>Reel Realities: How Gender and Age Shape Success Across Box Office and Streaming Platforms</h1>
</div>

### <u>Imports</u>

In [2]:
import pandas as pd

### 1. <u>Data cleaning and pre-processing</u>

#### 1.1 CMU Dataset

#### 1.2 IMDB Dataset

We will use three IMDB datasets to decribe movies:
1. "title.ratings.tsv" gives us the ratings of the movies as voted by viewers. 
2. "title.basics.tsv", indexes into "title.ratings.tsv" using a alphanumeric unique identifier of the title. It gives general information about the movie such as runtime, release date and adult rating.
3. "title.akas.tsv.gz" also indexes into the previous datasets with the same alphanumeric unique identifier. It gives more information about the movies such as language and region. Each dataset will be further detailed below.

In [13]:
# Loading the datasets. Null values are represented using "\N".
IMDB_ratings_df = pd.read_csv("data/IMDB/title.ratings.tsv", sep="\t", na_values="\\N")
IMDB_basics_df = pd.read_csv("data/IMDB/title.basics.tsv", sep="\t", na_values="\\N", low_memory=False)
IMDB_akas_df = pd.read_csv("data/IMDB/")

# Checking the number of null values in the dataset.
n_null_ratings = IMDB_ratings_df.isnull().sum()


IsADirectoryError: [Errno 21] Is a directory: 'data/IMDB/'

There are no null values. The dataset has the following attributes:
- tconst: alphanumeric unique identifier of the title
- averageRating: weighted average of all the individual user ratings
- numVotes: number of votes the title has received



### <u>Our success metric</u>

### 2. <u>Gender and age vs success BLABLABLA</u>

### 3. <u>How does it compare to streaming platforms? Are movies made for these platforms different? Have box office movies adapted since the rise of streaming?</u>

### 4. <u>What are the social reasons behind the presence of female characters in movies? Is it due to sexualization or genuine equality of representation?</u>

### 5. <u>Are these results different when looking at the acclaim (ratings) of a movie instead of revenue?</u>